# 🐦 Xeno-canto Downloader & CQT Analyzer

This notebook allows you to search for bird recordings on [Xeno-canto](https://www.xeno-canto.org/), download them, and perform a **Constant-Q Transform (CQT)** analysis.

### Features:
- **Search**: Query the Xeno-canto database by bird name (English or Scientific).
- **Download**: Automatically downloads the selected recording.
- **Analyze**: Visualizes the CQT (log-frequency spectrogram) which corresponds closely to musical notes.

In [ ]:
# Environment Setup
import sys
import subprocess

try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running on Google Colab")
    
    # Install dependencies
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gradio", "librosa", "soundfile", "matplotlib", "requests"], check=True)
    print("✅ Dependencies installed.")
except ImportError:
    IN_COLAB = False
    print("💻 Running locally")

In [ ]:
import gradio as gr
import requests
import os
import urllib.request
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

print("✅ Libraries imported.")

In [ ]:
def search_xenocanto(query):
    """Searches Xeno-canto API and returns a list of recordings."""
    if not query.strip():
        return gr.update(choices=[]), {}, "Please enter a search term."
        
    url = "https://www.xeno-canto.org/api/2/recordings"
    params = {'query': query}
    
    try:
        print(f"🔍 Searching for '{query}'...")
        response = requests.get(url, params=params)
        data = response.json()
        recordings = data['recordings']
        
        if not recordings:
            return gr.update(choices=[]), {}, "No recordings found."
            
        # Prepare choices for the dropdown
        choices = []
        metadata_store = {}
        
        # Limit to top 25 results to avoid overwhelming the UI
        for rec in recordings[:25]:
            # Format: English Name (Scientific Name) - ID - Country
            label = f"{rec['en']} ({rec['gen']} {rec['sp']}) - {rec['cnt']} [ID:{rec['id']}]"
            choices.append(label)
            metadata_store[label] = rec
            
        return gr.update(choices=choices, value=choices[0]), metadata_store, f"Found {len(recordings)} recordings. Showing top {len(choices)}."
        
    except Exception as e:
        return gr.update(choices=[]), {}, f"Error: {str(e)}"

def process_selection(selection, metadata_store):
    """Downloads the selected file and runs CQT analysis."""
    if not selection or selection not in metadata_store:
        return None, None, "Please select a recording first."
    
    rec = metadata_store[selection]
    file_url = rec['file']
    file_id = rec['id']
    # Create a safe filename
    safe_name = f"{rec['en']}_{rec['id']}".replace(" ", "_").replace("/", "-")
    file_name = f"{safe_name}.mp3"
    
    info_msg = ""
    
    # Download
    if not os.path.exists(file_name):
        try:
            print(f"⬇️ Downloading {file_name}...")
            urllib.request.urlretrieve(file_url, file_name)
            info_msg += f"Downloaded {file_name}. "
        except Exception as e:
            return None, None, f"Download failed: {str(e)}"
    else:
        info_msg += f"Using existing file {file_name}. "
        
    # Analyze
    try:
        fig = analyze_cqt(file_name)
        return file_name, fig, info_msg + "Analysis complete."
    except Exception as e:
        return file_name, None, info_msg + f"Analysis failed: {str(e)}"

def analyze_cqt(file_path):
    """Computes and plots the Constant-Q Transform."""
    # Load audio (limit duration to 60s to prevent memory issues on large files)
    y, sr = librosa.load(file_path, duration=60)
    
    # Compute CQT
    C = librosa.cqt(y, sr=sr)
    C_db = librosa.amplitude_to_db(np.abs(C), ref=np.max)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    img = librosa.display.specshow(C_db, sr=sr, x_axis='time', y_axis='cqt_note', ax=ax, cmap='magma')
    fig.colorbar(img, ax=ax, format='%+2.0f dB')
    ax.set_title(f"Constant-Q Transform (CQT)\n{os.path.basename(file_path)}")
    plt.tight_layout()
    
    return fig

In [ ]:
# Build the Gradio App
with gr.Blocks(title="Xeno-canto CQT Explorer") as demo:
    gr.Markdown("# 🐦 Xeno-canto CQT Explorer")
    gr.Markdown("Search for bird sounds, download them, and visualize their musical structure using CQT.")
    
    # State to store the full metadata of search results
    metadata_state = gr.State({})
    
    with gr.Row():
        with gr.Column(scale=1):
            search_input = gr.Textbox(label="Search Term", placeholder="e.g., Turdus rufiventris", value="Turdus rufiventris")
            search_btn = gr.Button("🔍 Search Xeno-canto", variant="primary")
            status_output = gr.Textbox(label="Status", interactive=False)
            
        with gr.Column(scale=2):
            # Dropdown is initially empty
            recording_dropdown = gr.Dropdown(label="Select Recording", choices=[], interactive=True)
            analyze_btn = gr.Button("⬇️ Download & Analyze", variant="secondary")
            
    with gr.Row():
        audio_player = gr.Audio(label="Audio Player", type="filepath")
        
    with gr.Row():
        cqt_plot = gr.Plot(label="CQT Analysis")

    # Event Handlers
    search_btn.click(
        fn=search_xenocanto,
        inputs=[search_input],
        outputs=[recording_dropdown, metadata_state, status_output]
    )
    
    analyze_btn.click(
        fn=process_selection,
        inputs=[recording_dropdown, metadata_state],
        outputs=[audio_player, cqt_plot, status_output]
    )

demo.launch(share=IN_COLAB)